In [109]:
import os

ruta_base = '../../'

# Cambiar el directorio de trabajo
os.chdir(ruta_base)

print("Directorio de trabajo actual:", os.getcwd())

Directorio de trabajo actual: /


In [ ]:
import pandas as pd
df_clima_ejemplo = pd.read_csv('processed/datos_clima/0200E_clima_completo.csv')

In [ ]:
# Tipos de datos de las columnas
print(df_clima_ejemplo.dtypes)

# Numerizar datos

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from datetime import datetime

In [ ]:
df_procesado = df_clima_ejemplo.copy()

### Procesar fechas

In [ ]:
# 1. Procesamiento de fechas y horas
# Convertir fecha a datetime y extraer características
df_procesado['fecha'] = pd.to_datetime(df_procesado['fecha'])
df_procesado['año'] = df_procesado['fecha'].dt.year
df_procesado['mes'] = df_procesado['fecha'].dt.month
df_procesado['dia'] = df_procesado['fecha'].dt.day
df_procesado['dia_semana'] = df_procesado['fecha'].dt.dayofweek
df_procesado['estacion_año'] = df_procesado['fecha'].dt.month % 12 // 3

### Procesar columnas de Horas y minutos

In [ ]:
print(df_procesado.columns)

In [ ]:
print(df_procesado['horaPresMax'].unique())
print(df_procesado['horaPresMin'].unique())

In [ ]:
print(df_procesado['horaracha'].unique())
print(df_procesado['horatmax'].unique())
print(df_procesado['horatmin'].unique())

In [ ]:
for col in ['horatmin', 'horatmax', 'horaracha', ]:
    if col in df_procesado.columns:
        # Ensure the column is of string type before applying the ~ operator
        df_procesado[col] = df_procesado[col].astype(str)
        print(col)
        print(df_procesado[col][~df_procesado[col].str.contains(':')].unique())
    else:
        print(f"La columna '{col}' no existe en el DataFrame.")

In [ ]:
def convertir_hora_a_minutos(x):
    if pd.isna(x) or x == 'nan':
        return pd.NA
    
    if x == 'Varias':
        # -1 podría ser un marcador para 'Varias'
        # (lo normalizaremos después, así que el -1 no afectará al modelo)
        return -1
        
    # Solo proceder si tiene el formato HH:MM esperado
    if isinstance(x, str) and ':' in x and x.replace(':', '').isdigit():
        try:
            horas, minutos = x.split(':')
            return int(horas) * 60 + int(minutos)
        except (ValueError, TypeError):
            return pd.NA
    else:
        return pd.NA

In [ ]:
# Convertir horas a valores numéricos con codificación especial
max_minutes = 24 * 60  # Total minutos en un día

for col in ['horatmin', 'horatmax', 'horaracha']:
    if col in df_procesado.columns:
        # Crear columna para indicar si es 'Varias'
        df_procesado[f'{col}_varias'] = (df_procesado[col] == 'Varias').astype(int)
        df_procesado[f'{col}_minutos'] = df_procesado[col].apply(convertir_hora_a_minutos)
        
        # Paso adicional: Normalizar entre 0 y 2pi para representación cíclica del tiempo
        # Filtrar solo los valores válidos (que no son NaN o -1)
        valid_mask = df_procesado[f'{col}_minutos'].notna() & (df_procesado[f'{col}_minutos'] != -1)
        valid_minutes = df_procesado.loc[valid_mask, f'{col}_minutos']
        
        # Crear las columnas seno y coseno para capturar la naturaleza cíclica del tiempo
        df_procesado[f'{col}_sin'] = np.nan
        df_procesado[f'{col}_cos'] = np.nan
        
        # Asignar los valores trigonométricos solo para los valores válidos
        if not valid_minutes.empty:
            # Convertir minutos a ángulos (0 a 2π)
            angles = valid_minutes.astype(float) * (2 * np.pi / max_minutes)
            df_procesado.loc[valid_mask, f'{col}_sin'] = np.sin(angles)
            df_procesado.loc[valid_mask, f'{col}_cos'] = np.cos(angles)

# Comprobar que las columnas se han creado correctamente
print("Columnas para horatmin:")
print(df_procesado[['horatmin_varias', 'horatmin_sin', 'horatmin_cos']].head())
print("\nColumnas para horatmax:")
print(df_procesado[['horatmax_varias', 'horatmax_sin', 'horatmax_cos']].head())
print("\nColumnas para horaracha:")
print(df_procesado[['horaracha_varias', 'horaracha_sin', 'horaracha_cos']].head())
        

### Procesar columnas de solo Horas

In [ ]:
def convertir_hora_entera(x):
    if pd.isna(x) or x == 'nan':
        return pd.NA
    
    if x == 'Varias':
        return -1
    
    try:
        # Convertir a entero (estas columnas solo tienen horas enteras)
        hora = int(x)
    
        # Manejar el caso especial de '24' (debería ser '00')
        if hora == 24:
            hora = 0
            
        # Convertir a minutos para mantener consistencia con otras columnas
        return hora * 60
    except (ValueError, TypeError):
        return pd.NA

In [ ]:
# Convertir horas de presión a valores numéricos (formato específico)
for col in ['horaPresMax', 'horaPresMin']:
    if col in df_procesado.columns:
        # Crear columna para indicar si es 'Varias'
        df_procesado[f'{col}_varias'] = (df_procesado[col] == 'Varias').astype(int)
        df_procesado[f'{col}_minutos'] = df_procesado[col].apply(convertir_hora_entera)

        # Normalización circular (valores entre 0 y 2π)
        valid_minutes = df_procesado[df_procesado[f'{col}_minutos'] >= 0][f'{col}_minutos']
        if not valid_minutes.empty:
            max_minutes = 24 * 60  # 24 horas * 60 minutos
            
            # Convertir a valores circulares
            df_procesado[f'{col}_sin'] = np.where(
                df_procesado[f'{col}_minutos'] >= 0,
                np.sin(2 * np.pi * df_procesado[f'{col}_minutos'] / max_minutes),
                0  # Valor por defecto para NA o 'Varias'
            )
            
            df_procesado[f'{col}_cos'] = np.where(
                df_procesado[f'{col}_minutos'] >= 0,
                np.cos(2 * np.pi * df_procesado[f'{col}_minutos'] / max_minutes),
                0  # Valor por defecto para NA o 'Varias'
            )
        
        # Eliminar la columna original y la intermedia
        df_procesado = df_procesado.drop(columns=[col, f'{col}_minutos'])

# Comprobar que las columnas se han creado correctamente
print("Columnas para horaPresMax:")
print(df_procesado[['horaPresMax_varias', 'horaPresMax_sin', 'horaPresMax_cos']].tail())
print("\nColumnas para horaPresMin:")
print(df_procesado[['horaPresMin_varias', 'horaPresMin_sin', 'horaPresMin_cos']].head())

### Procesar precipitación

In [ ]:
# Valores únicos en la columna 'precipitacion'
print(df_procesado['prec'].unique())

In [ ]:
def convertir_precipitacion(x):
        if pd.isna(x):
            return np.nan
        
        if isinstance(x, (int, float)):
            return float(x)
        
        if isinstance(x, str) and x == 'Ip':
            # Valor inapreciable: asignamos un valor pequeño (0.05 mm)
            return 0.05
            
        # Por si hay algún otro string inesperado
        try:
            return float(x)
        except (ValueError, TypeError):
            return np.nan

In [ ]:
# Procesamiento simplificado de la columna de precipitación
if 'prec' in df_procesado.columns:
    # Crear columna indicadora para precipitación inapreciable
    df_procesado['prec_inapreciable'] = df_procesado['prec'].apply(
        lambda x: 1 if isinstance(x, str) and x == 'Ip' else 0)
       
    # Aplicar la conversión
    df_procesado['prec_valor'] = df_procesado['prec'].apply(convertir_precipitacion)
    
    # Transformación logarítmica para manejar la naturaleza sesgada de las precipitaciones
    # Agregamos 1 para evitar log(0) y usar log(1+x)
    df_procesado['prec_log'] = np.log1p(df_procesado['prec_valor'])
    
    # Eliminamos la columna original
    df_procesado = df_procesado.drop(columns=['prec'])

# Comprobar que las columnas se han creado correctamente
print("Columnas para precipitación:")
print(df_procesado[['prec_inapreciable', 'prec_valor', 'prec_log']].head())

### Asegurar que las columnas float lo son

In [ ]:
# Asegurar que las columnas que deben ser de tipo float lo sean
# altitud, tmed, tmin, tmax, dir, velmedia, sol, presmax, presmin, hrmedia

# Verificar qué columnas existen en el dataframe
available_cols = []
for col in ['altitud', 'tmed', 'tmin', 'tmax', 'dir', 'velmedia', 'racha', 'sol', 'presMax', 'presMin', 'hrMedia']:
	if col in df_procesado.columns:
		available_cols.append(col)

# Convertir columnas a float si existen en el dataframe
for col in available_cols:
	try:
		df_procesado[col] = df_procesado[col].astype(float)
	except ValueError:
		# Si hay valores que no se pueden convertir (como comas en lugar de puntos)
		if isinstance(df_procesado[col][0], str) and ',' in df_procesado[col][0]:
			df_procesado[col] = df_procesado[col].str.replace(',', '.').astype(float)
		else:
			print(f"No se pudo convertir la columna {col} a tipo float.")

# Comprobar que las columnas se han convertido correctamente
print(df_procesado[available_cols].dtypes)

## Verificación de tipos de datos

In [ ]:
print(df_procesado.dtypes)


In [ ]:
print(df_procesado['racha'].unique())

In [ ]:
# Eliminar columnas que no se usarán en el modelo

# nombre                        object
# provincia                     object
# horatmin                      object
# horatmax                      object
# horaracha                     object

columnas_eliminar = ['nombre', 'provincia', 'horatmin', 'horatmax', 'horaracha']
df_procesado = df_procesado.drop(columns=columnas_eliminar)

print(df_procesado.columns)

### Guardar el dataframe

In [ ]:
!pwd

In [ ]:
# Guardar el dataframe procesado
ruta = 'data/processed/datos_clima/0200E_clima_Numerizado'
# df_procesado.to_parquet(ruta+".parquet", index=False)
df_procesado.to_csv(ruta+".parquet", index=False)

